In [ ]:
import numpy as np
import cv2 as cv
from matplotlib import pyplot as plt


# cv.namedWindow("img", cv.WINDOW_NORMAL)

# all reusable methodes

def show(img):
    cv.imshow("img", img)
    cv.waitKey(0)
    cv.destroyAllWindows()

def readShow(img):
    image_path = path(img)
    image = cv.imread(image_path)
    show(image)
    return image
    
def writeShow(img, name):
    image_path = path(name)
    cv.imwrite(image_path, img)
    show(img)

def path(name):
    return "./pictures/{}.png".format(name)


def neighbor(mat, i, j):
    arr = []
    arr.append(mat[i-1][j-1])
    arr.append(mat[i-1][j])
    arr.append(mat[i-1][j+1])

    arr.append(mat[i][j-1])
    arr.append(mat[i][j+1])

    arr.append(mat[i+1][j-1])
    arr.append(mat[i+1][j])
    arr.append(mat[i+1][j+1])
    return arr


# all reusalbe variables

img = readShow("image")

gray_img = cv.cvtColor(img, cv.COLOR_BGR2GRAY)




In [ ]:

# writeShow(gray_img, "gray_img")


In [ ]:
def better_mean_c(image, box_size):

    border_size = shift_size = int(box_size/2)
    mat = np.pad(image, border_size, 'symmetric')
    s = mat.shape

    for i in range(shift_size, s[0] - shift_size):
        for j in range(shift_size, s[1] - shift_size):

            box = mat[(i-border_size):(i+border_size+1),
                      (j-border_size):(j+border_size+1)]

            m = np.mean(box)
            std = np.std(box)
            # print(m, std)
            # mat[i, j] -= m
            k = 0.06
            R = 128
            t = m - k*std
            # t = m * (1 + k * (std/R - 1))
            # print(t)
            
            mat[i, j] = mat[i, j] <= t
            
            mat[i, j] *= 255
            
            # k = (np.amax(neighbor(box, 1, 1)) - np.amin(neighbor(box, 1, 1))) / \
            #     (np.amax(neighbor(box, 1, 1)) + np.amin(neighbor(box, 1, 1)) + 0.000001)
            # print(k)
            
    return mat

# better_mean_c(gray_img, 3)
# cv.imwrite(path("test"), better_mean_c(gray_img, 5))


In [ ]:
import enum
import math

class Threshold(enum.Enum):
    def default(subImg=[]):
        # m = np.mean(subImg)
        # i = subImg.shape[0]/2 - 1
        # j = subImg.shape[1]/2 - 1
        # print(i, j)
        # subImg[i, j] -= m
        return 127

    def niBlack(subImg=[], k = 0.06):
        m = np.mean(subImg)
        std = np.std(subImg)
        return m - k*std

    def sauvola(subImg=[], k=0.06, R=128):
        m = np.mean(subImg)
        std = np.std(subImg)
        return m * (1 + k * (std/R - 1))
    
    def brenser(subImg=[], k=0.06, R=128):
        imax = np.amax(subImg)
        imin = np.amin(subImg)
        return 0.5*(imin +imax)

    def nick(subImg=[], k=0.06, R=128):
        m = np.mean(subImg)
        n = subImg.shape[0] * subImg.shape[1]
        sum = np.sum(subImg**2)
        
        return m + k*(math.sqrt((sum - m**2)/n))
        
    def wolf(subImg=[], R=0, M=0, a=0.5):
        m = np.mean(subImg)
        std = np.std(subImg)
        return ((1-a)*m + a*M + ((a*std)/R)*(m-M))


def adaptiveThreshold(im, sizeI=4, sizeJ=4, k=1.0, type=Threshold.default):
    img = im.copy()

    s1 = img.shape[0]
    s2 = img.shape[1]

    sizeI = int(sizeI/2)
    sizeJ = int(sizeJ/2)

    if(sizeI <= 0):
        sizeI = 1

    if(sizeJ <= 0):
        sizeJ = 1
        
    if (type == Threshold.wolf):
        R = -1
        M = im[0, 0]
        for i in range(0, s1):
            for j in range(0, s2):
                
                posI1 = i-sizeI
                posI2 = i+sizeI

                posJ1 = j-sizeJ
                posJ2 = j+sizeJ

                if(posI1 < 0):
                    posI1 = 0

                if(posI2 >= s1):
                    posI2 = s1 - 1

                if(posJ1 < 0):
                    posJ1 = 0

                if(posJ2 >= s2):
                    posJ2 = s2 - 1
                    
                subImg = im[posI1:posI2, posJ1:posJ2]
                std = np.std(subImg)
                if (std > R):
                    R = std
                if (im[i, j] < M):
                    M = im[i, j]
        

    for i in range(0, s1):
        for j in range(0, s2):

            posI1 = i-sizeI
            posI2 = i+sizeI

            posJ1 = j-sizeJ
            posJ2 = j+sizeJ

            if(posI1 < 0):
                posI1 = 0

            if(posI2 >= s1):
                posI2 = s1 - 1

            if(posJ1 < 0):
                posJ1 = 0

            if(posJ2 >= s2):
                posJ2 = s2 - 1

            subImg = im[posI1:posI2, posJ1:posJ2]
            
            if (type == Threshold.wolf):
                img[i, j] = type(subImg, R, M)
            else:
                img[i, j] = type(subImg)

    t = img
    
    final_img = im.copy()
    final_img[final_img <= t] = 0
    final_img[final_img > t] = 255

    final_img[:, 0:9] = 0
    final_img[:, -9:] = 0
    final_img[0:9, :] = 0
    final_img[-9:, :] = 0
    
    return final_img







In [ ]:
def dilation(im, box_size):
    filter = np.ones((box_size, box_size))
    S = im.shape
    F = filter.shape
    image = im / 255
    padded_img = np.pad(image, 1, "symmetric")

    for i in range(S[0]):
        for j in range(S[1]):
            k = padded_img[i: i+F[0], j: j+F[1]]
            result = (k==filter)
            final = np.all(result == True)
            if final:
                image[i, j]  = 255
            else:
                image[i, j] = 0
    return image
                

In [ ]:
result = adaptiveThreshold(gray_img, 25, 25, 0.5, Threshold.default)
# writeShow(result, "multi")


In [ ]:
blured = cv.medianBlur(result, 3)
# writeShow(blured, "blured_mutli")

In [ ]:
final = dilation(blured, 3)
# writeShow(final, "dil_sauvola")


In [ ]:
hist, bins = np.histogram(gray_img.ravel(), 256, [0, 256])

plt.plot(hist)

In [ ]:

t = [140, 190]
def multi_thresh(img, t):
    im = img.copy()
    s = im.shape
    a = t[0]
    b = t[1]
    for i in range(s[0]):
        for j in range(s[1]):
            if (im[i, j] < a or im[i, j] > b):
                im[i, j] = 0
            else:
                im[i, j] = 255
    return im

im = multi_thresh(gray_img, t)

# show(im)
# writeShow(im, "multi")

In [ ]:
final1 = dilation(im, 7)
# final = dilation(im, 7)
# show(final1)
# writeShow(final1, "test3")



In [102]:

img = gray_img
a = 0
b = 255
n = 2  # number of thresholds (better choose even value)
k = 0.7  # free variable to take any positive value
T = []  # list which will contain 'n' thresholds


def multiThresh(img, a, b):
    if a > b:
        s = -1
        m = -1
        return m, s

    img = np.array(img)
    t1 = (img >= a)
    t2 = (img <= b)
    X = np.multiply(t1, t2)
    Y = np.multiply(img, X)
    s = np.sum(X)
    m = np.sum(Y)/s
    return m, s


for i in range(int(n/2-1)):
    img = np.array(img)
    t1 = (img >= a)
    t2 = (img <= b)
    X = np.multiply(t1, t2)
    Y = np.multiply(img, X)
    mu = np.sum(Y)/np.sum(X)

    Z = Y - mu
    Z = np.multiply(Z, X)
    W = np.multiply(Z, Z)
    sigma = math.sqrt(np.sum(W)/np.sum(X))

    T1 = mu - k*sigma
    T2 = mu + k*sigma

    x, y = multiThresh(img, a, T1)
    w, z = multiThresh(img, T2, b)

    T.append(x)
    T.append(w)

    a = T1+1
    b = T2-1
    k = k*(i+1)

T1 = mu
T2 = mu+1
x, y = multiThresh(img, a, T1)
w, z = multiThresh(img, T2, b)
T.append(x)
T.append(w)
T.sort()
im = multi_thresh(gray_img, T)

show(im)

QObject::moveToThread: Current thread (0x561adc2580c0) is not the object's thread (0x561adcb536f0).
Cannot move to target thread (0x561adc2580c0)

QObject::moveToThread: Current thread (0x561adc2580c0) is not the object's thread (0x561adcb536f0).
Cannot move to target thread (0x561adc2580c0)

QObject::moveToThread: Current thread (0x561adc2580c0) is not the object's thread (0x561adcb536f0).
Cannot move to target thread (0x561adc2580c0)

QObject::moveToThread: Current thread (0x561adc2580c0) is not the object's thread (0x561adcb536f0).
Cannot move to target thread (0x561adc2580c0)

QObject::moveToThread: Current thread (0x561adc2580c0) is not the object's thread (0x561adcb536f0).
Cannot move to target thread (0x561adc2580c0)

QObject::moveToThread: Current thread (0x561adc2580c0) is not the object's thread (0x561adcb536f0).
Cannot move to target thread (0x561adc2580c0)

QObject::moveToThread: Current thread (0x561adc2580c0) is not the object's thread (0x561adcb536f0).
Cannot move to tar

In [101]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np

from skimage import data
from skimage.filters import threshold_multiotsu



# thresholds = threshold_multiotsu(gray_img)
thresholds = [150, 180]

im = multi_thresh(gray_img, thresholds)

writeShow(im, "manual_multi")


QObject::moveToThread: Current thread (0x561adc2580c0) is not the object's thread (0x561adcb536f0).
Cannot move to target thread (0x561adc2580c0)

QObject::moveToThread: Current thread (0x561adc2580c0) is not the object's thread (0x561adcb536f0).
Cannot move to target thread (0x561adc2580c0)

QObject::moveToThread: Current thread (0x561adc2580c0) is not the object's thread (0x561adcb536f0).
Cannot move to target thread (0x561adc2580c0)

QObject::moveToThread: Current thread (0x561adc2580c0) is not the object's thread (0x561adcb536f0).
Cannot move to target thread (0x561adc2580c0)

QObject::moveToThread: Current thread (0x561adc2580c0) is not the object's thread (0x561adcb536f0).
Cannot move to target thread (0x561adc2580c0)

QObject::moveToThread: Current thread (0x561adc2580c0) is not the object's thread (0x561adcb536f0).
Cannot move to target thread (0x561adc2580c0)

QObject::moveToThread: Current thread (0x561adc2580c0) is not the object's thread (0x561adcb536f0).
Cannot move to tar

In [ ]:
blured = cv.GaussianBlur(im, (5, 5), 0)
show(blured)
# writeShow(blured, "blured_mutli")


In [ ]:
img = im


# Taking a matrix of size 5 as the kernel
kernel1 = np.ones((3, 3), np.uint8)
kernel2 = np.ones((7, 7), np.uint8)


# The first parameter is the origimorphologyExnal image,
# kernel is the matrix with which image is
# convolved and third parameter is the number
# of iterations, which will determine how much
# you want to erode/dilate a given image.
img_dilation = cv.dilate(img, kernel1, iterations=1)
img_erosion = cv.erode(img_dilation, kernel2, iterations=1)


show(img_erosion)
# show(img_dilation)

